In [1]:
%pip install groq python-dotenv --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.3/138.3 kB 10.1 MB/s eta 0:00:00


In [9]:
import os
from dotenv import load_dotenv
from groq import Groq
from getpass import getpass

load_dotenv()

# Prompt securely if API key is not set
if "GROQ_API_KEY" not in os.environ:
    os.environ["GROQ_API_KEY"] = getpass("Enter your GROQ API Key: ")

client = Groq(api_key=os.environ["GROQ_API_KEY"])

BASE_MODEL = "llama-3.1-8b-instant"

In [10]:
MODEL_CONFIG = {
    "technical": {
        "system_prompt": """You are a Technical Support Expert.
Be precise, code-focused, and rigorous.
Provide debugging steps and code examples when necessary.
Avoid unnecessary fluff."""
    },
    "billing": {
        "system_prompt": """You are a Billing Support Specialist.
Be empathetic, professional, and policy-driven.
Explain charges clearly and outline refund steps when applicable."""
    },
    "general": {
        "system_prompt": """You are a General Customer Support Assistant.
Be friendly, helpful, and conversational."""
    }
}

In [11]:
def route_prompt(user_input: str) -> str:
    """
    Classifies user query into: technical, billing, or general.
    Returns ONLY the category name.
    """

    routing_prompt = f"""
Classify the following text into one of these categories:
[technical, billing, general]

Return ONLY the category name.

Text:
{user_input}
"""

    response = client.chat.completions.create(
        model=BASE_MODEL,
        temperature=0.0,  # deterministic
        messages=[
            {"role": "user", "content": routing_prompt}
        ]
    )

    return response.choices[0].message.content.strip().lower()


In [12]:
def process_request(user_input: str) -> str:
    """
    Routes request and forwards to appropriate expert.
    Returns final expert response.
    """

    category = route_prompt(user_input)

    # Fallback safety
    if category not in MODEL_CONFIG:
        category = "general"

    system_prompt = MODEL_CONFIG[category]["system_prompt"]

    response = client.chat.completions.create(
        model=BASE_MODEL,
        temperature=0.7,  # creative/flexible for experts
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_input}
        ]
    )

    return response.choices[0].message.content


In [13]:
print("---- TECHNICAL EXAMPLE ----")
print(process_request("My python script is throwing an IndexError on line 5."))

print("\n---- BILLING EXAMPLE ----")
print(process_request("I was charged twice for my subscription this month."))

print("\n---- GENERAL EXAMPLE ----")
print(process_request("What are your business hours?"))

---- TECHNICAL EXAMPLE ----
To troubleshoot the issue, I'll need more information about your code. 

Please provide the following:

1. The code snippet that's causing the issue (at least the lines around line 5).
2. The entire error message, including the line and column numbers.

Here's a general example of how to debug an IndexError:

```python
# Example array
my_list = [1, 2, 3]

# Assuming line 5 is:
index = 5
try:
    value = my_list[index]
except IndexError as e:
    print(f"Error: {e}")
```

In this example, an IndexError will be raised because the index is out of range (it's 5, but the list only has 3 elements).

To resolve the issue, you can:

1. Check the index value to ensure it's within the valid range.
2. Verify that the list is not empty or has the expected elements.
3. Use list slicing or checks to avoid index-based access.

Provide more context about your code, and I'll be happy to help you debug the issue.

---- BILLING EXAMPLE ----
I'm so sorry to hear that you've bee